In [14]:
options(warn=-1)

In [15]:
library('edgeR')
library('limma')
library('dplyr')

In [16]:
p_val_thres <- 0.05

In [17]:
rds_path = '/tmp/work/RCproject/gene_lists.rds'
gene_lists<- readRDS(rds_path)
model_gene_list <- gene_lists$'de_intersect_plus_bulk_genes'
length(model_gene_list)

[1] 366

In [42]:
file_path = '/tmp/work/RCproject/GEO_singlecellexperiment.rds'
sce <- readRDS(file_path)
sce <- sce[, colData(sce)$Response != "partial"]

ncol(sce)

[1] 410

In [19]:
train_samples <- readLines("train_samples.txt")
head(train_samples)
sce <- sce[,train_samples]
sce

[1] "GSM5732588_GSE190826"  "GSM4523147_GSE150082"  "GSM6390460_GSE209746" 
[4] "GSM1103637_GSE45404-2" "GSM6390447_GSE209746"  "GSM5732586_GSE190826"

class: SingleCellExperiment 
dim: 12165 369 
metadata(0):
assays(3): counts logcounts scalelogcounts
rownames(12165): A1CF A2M ... ZZEF1 ZZZ3
rowData names(5): SYMBOL ENTREZID ENSEMBL gene_id gene_name
colnames(369): GSM5732588_GSE190826 GSM4523147_GSE150082 ...
  GSM5732574_GSE190826 GSM4304768_GSE145037
colData names(6): Response TRG ... Platform batch
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [34]:
table(colData(sce)$batch)

## [1] "GSE190826" "GSE209746"

## [1] "GSE133057"  "GSE145037"  "GSE150082"  "GSE45404-2" "GSE45404-1"
## [6] "GSE93375"   "GSE94104"


    GSE133057     GSE145037     GSE150082     GSE190826     GSE209746 
           22            30            27            89            88 
GSE45404_GPL1 GSE45404_GPL2      GSE93375      GSE94104 
           37            39            17            20 

In [43]:
all_data_sets <- sce
de4_data_sets <- sce[, colData(sce)$Platform %in% c("GPL13497","GPL14951","GPL15207","GPL6102")]
# de4_data_sets <- sce[, colData(sce)$batch %in% c("GSE133057","GSE145037","GSE150082","GSE45404-1","GSE45404-2","GSE93375","GSE94104")]
rna_seq_data_sets <- sce[, colData(sce)$batch %in% c("GSE190826","GSE209746")]

print('All')
table(colData(all_data_sets)$Response)
print('de4')
table(colData(de4_data_sets)$Response)
print('RNA_seq')
table(colData(rna_seq_data_sets)$Response)

[1] "All"



 no yes 
267 143 

[1] "de4"



 no yes 
 63  35 

[1] "RNA_seq"



 no yes 
148  53 

In [44]:
get_diff_genes <- function(input_sce){
    response <- colData(input_sce)$Response
    batch <- colData(input_sce)$batch
    print('current sample layout:')
    print(table(colData(input_sce)$batch))
    counts <- assay(input_sce, "scalelogcounts")
    batch <- factor(unlist(batch))
    response <- factor(unlist(response))
    plotMDS(counts, col = as.numeric(response))
    mm <- model.matrix(~0 + response + batch)
    fit <- lmFit(counts, mm)
    contr <- makeContrasts(responseyes - responseno, levels = colnames(coef(fit)))
    tmp <- contrasts.fit(fit, contr)
    tmp <- eBayes(tmp)
    top.table <- topTable(tmp, sort.by = "P", n = Inf)
    print(head(top.table, 20))
    print(length(which(top.table$P.Value < p_val_thres)))
    sig_genes <- rownames(top.table[top.table$P.Value < p_val_thres,])
    return(sig_genes)
}

In [45]:
DE_alldatasets <- get_diff_genes(all_data_sets)
DE_4datasets <- get_diff_genes(de4_data_sets)
DE_bulkDatasets <- get_diff_genes(rna_seq_data_sets)

In [ ]:
de_intersect_plus_bulk_genes <- intersect(union(DE_bulkDatasets,DE_4datasets),DE_alldatasets)
saveRDS(de_intersect_plus_bulk_genes,'ann_gene_set.rds')
saveRDS(DE_alldatasets,'ann_gene_set.rds')

In [ ]:
length(de_intersect_plus_bulk_genes)
length(model_gene_list)
length(intersect(model_gene_list,de_intersect_plus_bulk_genes))

In [25]:
#de_all should have 962 total...

[1] 366

[1] 224